# Azure RAG implementation using both Azure OpenAI and MaaP DeepSeek R1

## Setup
Follow instructions in https://github.com/Azure-Samples/azure-search-python-samples/blob/main/Tutorial-RAG/Tutorial-rag.ipynb to set up:
1. Azure Storage Account
2. Azure AI Search Service
3. Azure OpenAI Embedding Service
4. (Optional) Azure OpenAI GPT-4o model 

In [1]:
!pip install -r requirements.txt

In [2]:
!az account set -s f464793a-e174-43cd-b473-47ac97c91075

In [ ]:
# Set endpoints and API keys for Azure services
AZURE_SEARCH_SERVICE: str = "https://xxxx.search.windows.net"
AZURE_SEARCH_KEY: str = "xxxx"
AZURE_OPENAI_ACCOUNT: str = "https://xxxx.openai.azure.com"
AZURE_OPENAI_KEY: str = "xxxx"
AZURE_AI_MULTISERVICE_ACCOUNT: str = "https://xxxx.cognitiveservices.azure.com/"
AZURE_AI_MULTISERVICE_KEY: str = "xxxx"
AZURE_STORAGE_CONNECTION: str = "ResourceId=/subscriptions/xxxxx/resourceGroups/xxxxx/providers/Microsoft.Storage/storageAccounts/xxxxx5;"

# Example connection string for a search service managed identity connection:
# "ResourceId=/subscriptions/FAKE-SUBCRIPTION=ID/resourceGroups/FAKE-RESOURCE-GROUP/providers/Microsoft.Storage/storageAccounts/FAKE-ACCOUNT;

## Create the AI Search Index

In [4]:
from azure.core.credentials import AzureKeyCredential
credential = AzureKeyCredential(AZURE_SEARCH_KEY)

In [5]:

from azure.identity import DefaultAzureCredential
from azure.identity import get_bearer_token_provider
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    SearchIndex
)


# Create a search index  
index_name = "py-rag-tutorial-idx"
index_client = SearchIndexClient(endpoint=AZURE_SEARCH_SERVICE, credential=credential)  
fields = [
    SearchField(name="parent_id", type=SearchFieldDataType.String),  
    SearchField(name="title", type=SearchFieldDataType.String),
    SearchField(name="locations", type=SearchFieldDataType.Collection(SearchFieldDataType.String), filterable=True),
    SearchField(name="chunk_id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True, analyzer_name="keyword"),  
    SearchField(name="chunk", type=SearchFieldDataType.String, sortable=False, filterable=False, facetable=False),  
    SearchField(name="text_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), vector_search_dimensions=1024, vector_search_profile_name="myHnswProfile")
    ]  
  
# Configure the vector search configuration  
vector_search = VectorSearch(  
    algorithms=[  
        HnswAlgorithmConfiguration(name="myHnsw"),
    ],  
    profiles=[  
        VectorSearchProfile(  
            name="myHnswProfile",  
            algorithm_configuration_name="myHnsw",  
            vectorizer_name="myOpenAI",  
        )
    ],  
    vectorizers=[  
        AzureOpenAIVectorizer(  
            vectorizer_name="myOpenAI",  
            kind="azureOpenAI",  
            parameters=AzureOpenAIVectorizerParameters(  
                resource_url=AZURE_OPENAI_ACCOUNT,  
                deployment_name="text-embedding-3-small",
                model_name="text-embedding-3-small"
            ),
        ),  
    ], 
)  
  
# Create the search index
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search)  

In [6]:
print(index_client)

In [ ]:
result = index_client.create_or_update_index(index)  
print(f"{result.name} created")  

## Create the Data Source
Next step is to index the target dataset using Azure AI Search Indexer

In [7]:
from azure.search.documents.indexes import SearchIndexerClient
from azure.search.documents.indexes.models import (
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection
)

# Create a data source 
indexer_client = SearchIndexerClient(endpoint=AZURE_SEARCH_SERVICE, credential=credential)
container = SearchIndexerDataContainer(name="nasa-ebooks-pdfs-all")
data_source_connection = SearchIndexerDataSourceConnection(
    name="py-rag-tutorial-ds",
    type="azureblob",
    connection_string=AZURE_STORAGE_CONNECTION,
    container=container
)
data_source = indexer_client.create_or_update_data_source_connection(data_source_connection)

print(f"Data source '{data_source.name}' created or updated")

Data source 'py-rag-tutorial-ds' created or updated


## Create Indexer pipeline using Skills

In [8]:
from azure.search.documents.indexes.models import (
    SplitSkill,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    AzureOpenAIEmbeddingSkill,
    EntityRecognitionSkill,
    SearchIndexerIndexProjection,
    SearchIndexerIndexProjectionSelector,
    SearchIndexerIndexProjectionsParameters,
    IndexProjectionMode,
    SearchIndexerSkillset,
    CognitiveServicesAccountKey
)

# Create a skillset  
skillset_name = "py-rag-tutorial-ss"

split_skill = SplitSkill(  
    description="Split skill to chunk documents",  
    text_split_mode="pages",  
    context="/document",  
    maximum_page_length=2000,  
    page_overlap_length=500,  
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/content"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="textItems", target_name="pages")  
    ],  
)  
  
embedding_skill = AzureOpenAIEmbeddingSkill(  
    description="Skill to generate embeddings via Azure OpenAI",  
    context="/document/pages/*",  
    resource_url=AZURE_OPENAI_ACCOUNT,  
    deployment_name="text-embedding-3-small",  
    model_name="text-embedding-3-small",
    dimensions=1024,
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/pages/*"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="embedding", target_name="text_vector")  
    ],  
)

entity_skill = EntityRecognitionSkill(
    description="Skill to recognize entities in text",
    context="/document/pages/*",
    categories=["Location"],
    default_language_code="en",
    inputs=[
        InputFieldMappingEntry(name="text", source="/document/pages/*")
    ],
    outputs=[
        OutputFieldMappingEntry(name="locations", target_name="locations")
    ]
)
  
index_projections = SearchIndexerIndexProjection(  
    selectors=[  
        SearchIndexerIndexProjectionSelector(  
            target_index_name=index_name,  
            parent_key_field_name="parent_id",  
            source_context="/document/pages/*",  
            mappings=[  
                InputFieldMappingEntry(name="chunk", source="/document/pages/*"),  
                InputFieldMappingEntry(name="text_vector", source="/document/pages/*/text_vector"),
                InputFieldMappingEntry(name="locations", source="/document/pages/*/locations"),  
                InputFieldMappingEntry(name="title", source="/document/metadata_storage_name"),  
            ],  
        ),  
    ],  
    parameters=SearchIndexerIndexProjectionsParameters(  
        projection_mode=IndexProjectionMode.SKIP_INDEXING_PARENT_DOCUMENTS  
    ),  
) 

cognitive_services_account = CognitiveServicesAccountKey(key=AZURE_AI_MULTISERVICE_KEY)

skills = [split_skill, embedding_skill, entity_skill]

skillset = SearchIndexerSkillset(  
    name=skillset_name,  
    description="Skillset to chunk documents and generating embeddings",  
    skills=skills,  
    index_projection=index_projections,
    cognitive_services_account=cognitive_services_account
)
  
client = SearchIndexerClient(endpoint=AZURE_SEARCH_SERVICE, credential=credential)  
client.create_or_update_skillset(skillset)  
print(f"{skillset.name} created")  

py-rag-tutorial-ss created


## Create the Indexer

In [9]:
from azure.search.documents.indexes.models import (
    SearchIndexer
)

# Create an indexer  
indexer_name = "py-rag-tutorial-idxr-1" 

indexer_parameters = None

indexer = SearchIndexer(  
    name=indexer_name,  
    description="Indexer to index documents and generate embeddings",  
    skillset_name=skillset_name,  
    target_index_name=index_name,  
    data_source_name=data_source.name,
    parameters=indexer_parameters
)  

# Create and run the indexer  
indexer_client = SearchIndexerClient(endpoint=AZURE_SEARCH_SERVICE, credential=credential)  
indexer_result = indexer_client.create_or_update_indexer(indexer)  

print(f' {indexer_name} is created and running. Give the indexer a few minutes before running a query.')  

 py-rag-tutorial-idxr-1 is created and running. Give the indexer a few minutes before running a query.


## Test the resulting AI Search Index

In [10]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery

# Vector Search using text-to-vector conversion of the query string
query = "what's NASA's website?"  

search_client = SearchClient(endpoint=AZURE_SEARCH_SERVICE, credential=credential, index_name=index_name)
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="text_vector")
  
results = search_client.search(  
    search_text=query,  
    vector_queries= [vector_query],
    select=["chunk"],
    top=1
)  
  
for result in results:  
    print(f"Score: {result['@search.score']}")
    print(f"Chunk: {result['chunk']}")

Score: 0.01666666753590107
Chunk: national Aeronautics and Space Administration

earth Science

NASA Headquarters 

300 E Street SW 

Washington, DC 20546

www.nasa.gov

np-2018-05-2546-hQ


## Set up AzureOpenAI client

In [11]:
from openai import AzureOpenAI

In [12]:
aoai_credential = AzureKeyCredential(AZURE_OPENAI_KEY)

In [13]:
search_credential = AzureKeyCredential(AZURE_SEARCH_KEY)

In [14]:
default_credential = DefaultAzureCredential()

In [15]:
# Import libraries
from azure.search.documents import SearchClient

# Set up the Azure OpenAI client
token_provider = get_bearer_token_provider(default_credential, "https://cognitiveservices.azure.com/.default")
openai_client = AzureOpenAI(
     api_version="2024-08-01-preview",
     azure_endpoint=AZURE_OPENAI_ACCOUNT,
     azure_ad_token_provider=token_provider
 )

deployment_name = "gpt4o-helen"

# Set up the Azure Azure AI Search client
search_client = SearchClient(
     endpoint=AZURE_SEARCH_SERVICE,
     index_name=index_name,
     credential=search_credential
 )

# Provide instructions to the model
GROUNDED_PROMPT="""
You are an AI assistant that helps users learn from the information found in the source material.
Answer the query using only the sources provided below.
Use bullets if the answer has multiple points.
If the answer is longer than 3 sentences, provide a summary.
Answer ONLY with the facts listed in the list of sources below. Cite your source when you answer the question
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Query: {query}
Sources:\n{sources}
"""

## Execute the "Retriever" RAG component

In [16]:
# Provide the search query. 
# It's hybrid: a keyword search on "query", with text-to-vector conversion for "vector_query".
# The vector query finds 50 nearest neighbor matches in the search index
query="What's the NASA earth book about?"
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="text_vector")

# Set up the search results and the chat thread.
# Retrieve the selected fields from the search index related to the question.
# Search results are limited to the top 5 matches. Limiting top can help you stay under LLM quotas.
search_results = search_client.search(
    search_text=query,
    vector_queries= [vector_query],
    select=["title", "chunk", "locations"],
    top=5,
)

print(search_results)

<iterator object azure.core.paging.ItemPaged at 0x29fd993ead0>


In [17]:
# Newlines could be in the OCR'd content or in PDFs, as is the case for the sample PDFs used for this tutorial.
# Use a unique separator to make the sources distinct. 
# We chose repeated equal signs (=) followed by a newline because it's unlikely the source documents contain this sequence.
sources_formatted = "=================\n".join([f'TITLE: {document["title"]}, CONTENT: {document["chunk"]}, LOCATIONS: {document["locations"]}' for document in search_results])

## Execute the "Generator" RAG component using AOAI

In [18]:
response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": GROUNDED_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=deployment_name
)

print(response.choices[0].message.content)

The NASA Earth book is a work that lies at the intersection of science and art. It explores how NASA has studied Earth's physical processes from beneath its crust to the edge of the atmosphere, using innovative tools to examine the water cycle, carbon cycle, ocean circulation, and the movement of heat. The book features inspiring images that tell the story of a 4.5-billion-year-old planet, showcasing the interconnected dance of land, wind, water, ice, and air as viewed from space. 

The book emphasizes NASA's unique vantage point in observing and understanding Earth, while also celebrating the planet's inherent beauty and complexity through a collection of vivid, scientifically accurate images (source: page-8.pdf).


## Execute the "Generator" RAG component using DeepSeek

In [ ]:
# If using my own subscription then execute the below block.
from openai import OpenAI
client = OpenAI(
    base_url="https://r1-prod-helen.centralus.inference.ml.azure.com/v1",
    api_key="xxxx"
)

completion = client.chat.completions.create(
  model="deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
  messages=[
    {"role": "user", "content": GROUNDED_PROMPT.format(query=query, sources=sources_formatted)}
  ],
  stream=True
)

for chunk in completion:
  print(chunk.choices[0].delta.content, end='')

Okay, so I'm trying to figure out what the NASA Earth Book is about. Let me look through the sources provided. 

Starting with page-8.pdf, it says the book stands at the intersection of science and art. It talks about NASA studying Earth in novel ways using tools to look at physical processes, from beneath the crust to the atmosphere. It mentions looking at Earth in macrocosm and microcosm, like mountain streams and jet streams. It also says they examine Earth as a system, looking at cycles like the water and carbon cycles, ocean circulation, and heat movement. They measure particles, gases, energy, and fluids, and study light—how it reflects, refracts, etc. It's like they're taking a step back to admire Earth's beauty and see it as a jewel from space. The images chosen are meant to inspire, showing the planet's diversity and that no human imagination can match its real beauty.

Moving to page-175.pdf, it's about the authors. Michael Carlowicz is the managing editor, and Kathy Carroll 